# 0: Download Data Section

In [ ]:
# Prepare for using kaggle api
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Download data
!pip install kaggle
!kaggle competitions download -c digit-recognizer

Mounted at /content/drive/
 98% 9.00M/9.16M [00:00<00:00, 62.8MB/s]
100% 9.16M/9.16M [00:00<00:00, 58.8MB/s]
  0% 0.00/6.09M [00:00<?, ?B/s]
100% 6.09M/6.09M [00:00<00:00, 95.6MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 75.6MB/s]


# 1.Prepare section

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import ToTensor
from torch import nn, optim
import tqdm
from collections import OrderedDict
from torch.utils.data.sampler import (BatchSampler, RandomSampler)

batch_size = 256

# 2. Transform Section

In [ ]:
class ComposeTransform():
    """
    複数のTransformをまとめあげる
    """
    def __init__(self, transforms=None):
        """
        Parameters
        --------------
        transforms: list
            transformのインスタンスをリストにして渡す
        """
        self.transforms = transforms

    def __call__(self, x):
        if self.transforms:
            for transform in self.transforms:
                x = transform(x)
        return x

class BaseTransform():
    """
    自作Transformの基底クラス
    """
    def __init__(self, debug=False):
        self.debug = debug
    
    def __call__(self):
        raise NotImplementedError()

class SimpleTransform(BaseTransform):
    """
    とりあえずのクラス
    よく使うものを入れておく
    扱う関数が増えてきたらテーマごとに分離する
    """
    def __call__(self, x):
        if self.debug:
            # ここで途中途中のxの値を確認できるようにしたい
            pass
        x = x.reshape(28, 28, 1)
        x = x.astype('uint8')

        return x

# 3. Dataset Section

In [ ]:
class DigitDataset():
    def __init__(self, path, transform= None, train=True):
        self.transform = transform
        # pandasは処理が遅いのでなるべく早くnumpyへ
        self.data = pd.read_csv(path).values
        self.train = train
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if self.train:
            label =self.data[idx, 0]
            x = self.data[idx, 1:]
            if self.transform:
                x = self.transform(x)
                label = torch.tensor(label).squeeze()
            return x, label
        else:
            x = self.data[idx, :]
            if self.transform:
                x = self.transform(x)
            return x

# 4. Model Section

In [ ]:
class FlattenLayer(nn.Module):
    """
    (N,C,H,W) -> (N,C*H*W)
    """
    def forward(self, x):
        size = x.size()
        return x.view(size[0], -1)

conv_net = nn.Sequential(
    nn.Conv2d(1, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    FlattenLayer()
)


test_input = torch.ones(1,1,28,28)
conv_output_size = conv_net(test_input).size()[-1]

mlp_net = nn.Sequential(
    nn.Linear(conv_output_size, 200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout2d(0.25),
    nn.Linear(200, 10) 
)

net = nn.Sequential(
    conv_net,
    mlp_net
)

# 5.Main function section

In [ ]:
def train_net(net, train_loader, eval_loader, optim_cls=optim.SGD, loss_fn=nn.CrossEntropyLoss(), n_iter=20, device= 'cuda:0'):
    optimizer = optim_cls(net.parameters(), lr=0.1)
    losses = []
    val_losses = []
    train_acc = []
    val_acc = []
    n = 0
    n_acc = 0
    net = net.to(device)

    for epoch in range(n_iter):
        running_loss=0.0
        net.train()
        with tqdm.tqdm(train_loader) as pbar:
            for i, (x, label) in enumerate(pbar):
                x = x.to(device)
                label = label.to(device)
                h = net(x)
                loss = loss_fn(h, label)
                running_loss+=loss.item()
                n += len(label)
                _, y_pred = h.max(1)
                n_acc += (y_pred==label).float().sum().item()

                # 逆伝播によるパラメータ更新
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                pbar.set_postfix(OrderedDict(
                    epoch= epoch+1,
                    loss=running_loss/(i+1), 
                    ))
            losses.append(running_loss / len(train_loader))
            train_acc.append(n_acc / n)
            val_loss, val_acc_ = val_net(net, eval_loader, loss_fn)
            val_losses.append(val_loss)
            val_acc.append(val_acc_)

    return losses, val_losses, train_acc, val_acc

def val_net(net, eval_loader, loss_fn, device= 'cuda:0'):
    net.eval()
    val_acc = 0
    val_loss = 0
    n = 0
    n_acc =0
    running_loss=0.0
    net = net.to(device)
    for i, (x, label) in enumerate(val_loader):
        x = x.to(device)
        label = label.to(device)
        h = net(x)
        loss = loss_fn(h, label)
        running_loss+=loss.item()
        n += len(label)
        _, y_pred = h.max(1)
        n_acc += (y_pred==label).float().sum().item()
    val_acc = n_acc / n
    val_loss = running_loss / len(val_loader)
    return val_loss, val_acc

def pred_net(net, test_loader, device= 'cuda:0'):
    y_preds = []
    net = net.to(device)
    for i, x in enumerate(test_loader):
        x = x.to(device)
        h = net(x)
        _, y_pred = h.max(1)
        y_preds.append(y_pred)
    return torch.cat(y_preds,dim=0)

# 6. Train Section

In [ ]:
transform_1 = SimpleTransform()
transform_2 = ToTensor()
transform = ComposeTransform([transform_1, transform_2])
dataset = DigitDataset('/content/train.csv.zip', transform)
train_size = int(len(dataset)*0.8)
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
losses, val_losses, train_acc, val_acc= train_net(net, train_loader, train_loader)

100%|██████████| 132/132 [00:02<00:00, 60.60it/s, epoch=20, loss=0.0129]


# 7. Validation Section

In [1]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.plot(val_losses)
plt.show()
print(train_acc)
print(val_acc)

NameError: ignored

# 8. Test Section

In [ ]:
test_dataset = DigitDataset('/content/test.csv.zip', transform, train=False)
test_loader = DataLoader(test_dataset, batch_size= batch_size)
y_preds = pred_net(net, test_loader)
submission_df = pd.read_csv('/content/sample_submission.csv', index_col='ImageId')
y_pred_numpy = np.array(y_preds.to('cpu'))
submission_df['Label'] = y_pred_numpy
submission_df
submission_df.to_csv("submission.csv")